# 导入

In [1]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP

d:\softwares\python\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\softwares\python\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\softwares\python\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument was

# 加载数据

In [2]:
# step1 加载文件
docs = [
  '今天天气很好',
  '今天天气很不错',
  '今天天气晴朗',
  '今天天气不错',
  '今天阳光明媚',
  '自然语言处理会使用到深度学习',
  '自然语言处理会使用到线性代数',
  '自然语言处理会需要掌握一定的微积分知识',
  '自然语言处理很简单',
  '自然语言处理容易掌握'
]

# 创建词向量模型

In [3]:
# 方式1：从网络中下载
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 方式2：从本地加载，注意\\转义
# 路径写到包含config.json的位置
# embedding_model = SentenceTransformer('C:\\Users\\ji\\.cache\\huggingface\\hub\\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2\\snapshots\\bf3bf13ab40c3157080a7ab344c831b9ad18b5eb')
embedding_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

# 训练词向量

In [4]:
embeddings = embedding_model.encode(docs, show_progress_bar=True)
print(type(embeddings), embeddings.shape)
print(embeddings)
# 这里生成的六个语义向量，对前面的六个句子

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.56it/s]

<class 'numpy.ndarray'> (10, 384)
[[ 0.22831464  0.20791563 -0.01361518 ...  0.08457322 -0.27376777
   0.30211884]
 [ 0.23415914  0.18763618 -0.00334522 ...  0.07795675 -0.28823084
   0.29783475]
 [ 0.25473982  0.1818737   0.02977072 ...  0.12291708 -0.15997928
   0.20394088]
 ...
 [-0.10149594  0.03836722 -0.28412744 ...  0.23825662 -0.04146777
  -0.19284798]
 [-0.28807735 -0.02647653  0.18795004 ...  0.48011842  0.34286204
  -0.21912694]
 [-0.1399834  -0.17367244 -0.00831866 ...  0.33865795  0.19647521
  -0.17728156]]


# 文本聚类 ⭐

In [5]:
# 创建模型
topic_model = BERTopic(
  embedding_model=embedding_model,
  min_topic_size=2, # 重要参数
  verbose=True,
)

# 训练模型
topic_model.fit_transform(docs, embeddings=embeddings)

# 获取聚类结果
topic_info = topic_model.get_topic_info()
topic_info

2024-04-19 09:34:02,442 - BERTopic - Reduced dimensionality
2024-04-19 09:34:02,446 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name,Representation,Representative_Docs
0,0,5,0_今天阳光明媚_今天天气晴朗_今天天气很好_今天天气很不错,"[今天阳光明媚, 今天天气晴朗, 今天天气很好, 今天天气很不错, 今天天气不错, , , ...","[今天天气晴朗, 今天天气很不错, 今天天气很好]"
1,1,5,1_自然语言处理很简单_自然语言处理容易掌握_自然语言处理会需要掌握一定的微积分知识_自然语...,"[自然语言处理很简单, 自然语言处理容易掌握, 自然语言处理会需要掌握一定的微积分知识, 自...","[自然语言处理容易掌握, 自然语言处理会需要掌握一定的微积分知识, 自然语言处理会使用到线性代数]"


# 可视化

In [6]:
# ⭐ UMAP可视化
reduced_embeddings = UMAP(n_neighbors=5, n_components=2, min_dist=0.0, metric='cosine',).fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_annotations=True)